In [1]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '442final'

In [3]:
import os
import sys

GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
sys.path.append(GOOGLE_DRIVE_PATH)


In [4]:
print(GOOGLE_DRIVE_PATH)

drive/MyDrive/442final


In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
#navigate to folder
%cd /content/drive/MyDrive/442final
!nvidia-smi


/content/drive/MyDrive/442final
Sun Apr 28 21:50:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   34C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------

In [ ]:
#unzip data and then step back out
%cd data
!unzip datazip
!ls
%cd ..

In [7]:
!ls data/archive

 0    12   16   2    23   27   30   34	 38   41   45   49   52   56   6    8		 +_sampled
 1    13   17   20   24   28   31   35	 39   42   46   5    53   57   60   9		 -_sampled
 10   14   18   21   25   29   32   36	 4    43   47   50   54   58   61  '(_sampled'	 x_sampled
 11   15   19   22   26   3    33   37	 40   44   48   51   55   59   7   ')_sampled'	 Ў_sampled


In [ ]:
import os
import shutil

##make renamed directory, code from Yuvraj Joshi
##I will be using this modified dataset to train my models

def rename_and_copy_folders(root_dir, dest_dir):
    # Define mapping for renaming specific items
    mapping = {
        '+_sampled': '+',
        '-_sampled': '-',
        '(_sampled': '(',
        ')_sampled': ')',
        'x_sampled': '*',
        'Ў_sampled': '÷',
        "10": "A", "11": "B", "12": "C", "13": "D", "14": "E", "15": "F",
        "16": "G", "17": "H", "18": "I", "19": "J", "20": "K", "21": "L",
        "22": "M", "23": "N", "24": "O", "25": "P", "26": "Q", "27": "R",
        "28": "S", "29": "T", "30": "U", "31": "V", "32": "W", "33": "X",
        "34": "Y", "35": "Z", "36": "a", "37": "b", "38": "c", "39": "d",
        "40": "e", "41": "f", "42": "g", "43": "h", "44": "i", "45": "j",
        "46": "k", "47": "l", "48": "m", "49": "n", "50": "o", "51": "p",
        "52": "q", "53": "r", "54": "s", "55": "t", "56": "u", "57": "v",
        "58": "w", "59": "x", "60": "y", "61": "z",
    }
    for subdir, dirs, files in os.walk(root_dir):
        # Get the name of the current directory
        dir_name = os.path.basename(subdir)
        # Get the new name from mapping
        new_name = mapping.get(dir_name, dir_name)
        # Construct the new directory path
        new_dir_path = os.path.join(dest_dir, new_name)
        # Create the new directory
        os.makedirs(new_dir_path, exist_ok=True)
        # Copy files to the new directory, taking only every 10th file
        for i, file in enumerate(files):
            if i % 10 == 0:  # This checks if the file index is a multiple of 10
                src_file_path = os.path.join(subdir, file)
                print("source file path: ", src_file_path)

                dest_file_path = os.path.join(new_dir_path, file)
                print("dest file path: ", dest_file_path)
                shutil.copyfile(src_file_path, dest_file_path)

# Specify data paths
root_directory = "/content/drive/MyDrive/442final/data/archive"
destination_directory = "/content/drive/MyDrive/442final/renamed"
print("call")
rename_and_copy_folders(root_directory, destination_directory)


In [11]:
!ls renamed


'('   +   0   3   6   9   b   C   e   F   h   I   k   L   n   O   q   R   t   U   w   X   z
')'   -   1   4   7   a   B   d   E   g   H   j   K   m   N   p   Q   s   T   v   W   y   Z
'*'   ÷   2   5   8   A   c   D   f   G   i   J   l   M   o   P   r   S   u   V   x   Y


In [12]:
!ls


442final.ipynb						    mobilenetv3_symbol_classifier_fold_5.h5
data							    mobilenetv3_symbol_classifier_tuned.h5
mobilenetv3_symbol_classifier_2.h5			    mobilenetv3_symbol_cv_tuned.h5
mobilenetv3_symbol_classifier_3.h5			    my_dir
mobilenetv3_symbol_classifier_5.h5			    renamed
mobilenetv3_symbol_classifier_6_res_dropout_every_layer.h5  renamed_data.zip
mobilenetv3_symbol_classifier_6_res.h5			    test_set
mobilenetv3_symbol_classifier_7.h5


In [8]:
##imports for ML
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay


In [ ]:
##My first morel, M2a

# Define the root directory of your dataset
dataset_dir = "/content/drive/MyDrive/442final/renamed"

# Define the image size and input shape
img_height, img_width = 64, 64  # Updated to match the input shape of the model
input_shape = (img_height, img_width, 1)  # Assuming grayscale images

# Prepare data with augmented data generation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    color_mode='grayscale')  # Ensure grayscale mode if images are grayscale

val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    color_mode='grayscale')  # Ensure grayscale mode if images are grayscale

num_classes = 68  # 10 digits + 4 symbols (+, -, ×, ÷) + 2 parentheses ( )

# Build the CNN model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
initial_learning_rate = 0.001  # Starting with a larger learning rate

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()
# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator
)

# Save the trained model
model.save("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_2.h5")




In [ ]:
# Function to predict class for a single image
def predict_class(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width), color_mode='grayscale')
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction)
    class_labels = sorted(train_generator.class_indices.keys(), key=lambda x: train_generator.class_indices[x])
    predicted_class = class_labels[predicted_class_index]
    return predicted_class

# Example usage: Predicting class for a single image
image_path = "/content/drive/MyDrive/442final/renamed/0/3637.png"  # Adjust with your image path
predicted_class = predict_class(image_path, model)
print("Predicted class:", predicted_class)

In [ ]:
##make a test set to compare all of my models
##saved in directory test_set
import os
import shutil

def create_test_set(root_dir, dest_dir):
    # Define mapping for renaming specific items
    mapping = {
        '+_sampled': '+',
        '-_sampled': '-',
        '(_sampled': '(',
        ')_sampled': ')',
        'x_sampled': '*',
        'Ў_sampled': '÷',
        "10": "A", "11": "B", "12": "C", "13": "D", "14": "E", "15": "F",
        "16": "G", "17": "H", "18": "I", "19": "J", "20": "K", "21": "L",
        "22": "M", "23": "N", "24": "O", "25": "P", "26": "Q", "27": "R",
        "28": "S", "29": "T", "30": "U", "31": "V", "32": "W", "33": "X",
        "34": "Y", "35": "Z", "36": "a", "37": "b", "38": "c", "39": "d",
        "40": "e", "41": "f", "42": "g", "43": "h", "44": "i", "45": "j",
        "46": "k", "47": "l", "48": "m", "49": "n", "50": "o", "51": "p",
        "52": "q", "53": "r", "54": "s", "55": "t", "56": "u", "57": "v",
        "58": "w", "59": "x", "60": "y", "61": "z",
    }
    for subdir, dirs, files in os.walk(root_dir):
        # Get the name of the current directory
        dir_name = os.path.basename(subdir)
        # Get the new name from mapping
        new_name = mapping.get(dir_name, dir_name)
        # Construct the new directory path
        new_dir_path = os.path.join(dest_dir, new_name)
        # Create the new directory
        os.makedirs(new_dir_path, exist_ok=True)
        # Copy files to the new directory, taking only every 10th file
        for i, file in enumerate(files):
           if i % 21 == 0 and i % 10 != 0:
                src_file_path = os.path.join(subdir, file)
                print("source file path: ", src_file_path)

                dest_file_path = os.path.join(new_dir_path, file)
                print("dest file path: ", dest_file_path)
                shutil.copyfile(src_file_path, dest_file_path)

# Specify data paths
root_directory = "/content/drive/MyDrive/442final/data/archive"
destination_directory = "/content/drive/MyDrive/442final/test_set"
print("call")
create_test_set(root_directory, destination_directory)



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_2.h5")
model.summary()

# Set up the data generator for the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Ensure the generator matches the model input requirements
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/442final/test_set',
    target_size=(64, 64),  # Match the input size for which the model is configured
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale',
    shuffle=False
)

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        320       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 16, 16, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 8, 8, 128)        

In [ ]:
#model architecture from Yuvrav Joshi
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Reshape
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.optimizers import Adam

# Assuming you have already imported the necessary modules
# Define the root directory of your dataset
dataset_dir = "/content/drive/MyDrive/442final/renamed"

# Define the image size and input shape
img_height, img_width = 64, 64
input_shape = (img_height, img_width, 1)  # Grayscale images

# Prepare data with augmented data generation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    color_mode='grayscale')

val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    color_mode='grayscale')

# Model configuration
num_classes = 68

# Build the CNN model inspired by modelB but adapted for your dataset
model = Sequential([
    Conv2D(32, (5, 5), activation='relu', padding='same', input_shape=input_shape, kernel_constraint=MaxNorm(3)),
    Conv2D(32, (5, 5), activation='relu', kernel_constraint=MaxNorm(3)),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(1024, activation='relu', kernel_constraint=MaxNorm(3)),
    Dense(512, activation='relu', kernel_constraint=MaxNorm(3)),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator
)

# Save the trained model
model.save("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_3.h5")

# Function to predict class for a single image
def predict_class(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width), color_mode='grayscale')
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction)
    class_labels = sorted(train_generator.class_indices.keys(), key=lambda x: train_generator.class_indices[x])
    predicted_class = class_labels[predicted_class_index]
    return predicted_class

# Example usage: Predicting class for a single image
image_path = "/content/drive/MyDrive/442final/renamed/0/3637.png"  # Adjust with your image path
predicted_class = predict_class(image_path, model)
print("Predicted class:", predicted_class)


Found 6120 images belonging to 68 classes.
Found 1496 images belonging to 68 classes.
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 64, 64, 32)        832       
                                                                 
 conv2d_9 (Conv2D)           (None, 60, 60, 32)        25632     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 30, 30, 32)        0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 28800)             0         
                                                                 
 dense_7 (Dense)             (None, 1024)              29492224  
                                                                 
 dense_8 (Dense)             (None

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 115ms/step
Predicted class: 0


In [ ]:
#test Joshi's CNN
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_3.h5")
model.summary()

# Set up the data generator for the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Ensure the generator matches the model input requirements
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/442final/test_set',
    target_size=(64, 64),  # Match the input size for which the model is configured
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' because your model uses categorical crossentropy
    color_mode='grayscale',  # Match the color mode to how the model was trained
    shuffle=False  # Set to False to keep data in the same order for consistency
)

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 64, 64, 32)        832       
                                                                 
 conv2d_9 (Conv2D)           (None, 60, 60, 32)        25632     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 30, 30, 32)        0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 28800)             0         
                                                                 
 dense_7 (Dense)             (None, 1024)              29492224  
                                                                 
 dense_8 (Dense)             (None, 512)               524800    
                                                      

In [ ]:
#M2a with L2 regularization and increased dropout rate.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the root directory of your dataset
dataset_dir = "/content/drive/MyDrive/442final/renamed"

# Define the image size and input shape
img_height, img_width = 64, 64
input_shape = (img_height, img_width, 1)  # Assuming grayscale images

# Prepare data with augmented data generation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    color_mode='grayscale')

val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    color_mode='grayscale')

num_classes = 68

# Build the CNN model with increased dropout and L2 regularization
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape, kernel_regularizer=l2(0.01)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),  # Increased dropout rate
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    epochs=30,  #last time pleatued around 20,
    validation_data=val_generator
)

# Save the trained model
model.save("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_4.h5")

##ended this process early because it showed low accuracy and a slow growth trend


Found 6120 images belonging to 68 classes.
Found 1496 images belonging to 68 classes.
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 64, 64, 32)        320       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 16, 16, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_12 (Conv2D)          (None, 16, 16, 128)       73856     
                                  

KeyboardInterrupt: 

In [ ]:

#L2 regularization only
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the root directory of your dataset
dataset_dir = "/content/drive/MyDrive/442final/renamed"

# Define the image size and input shape
img_height, img_width = 64, 64
input_shape = (img_height, img_width, 1)  # Assuming grayscale images

# Prepare data with augmented data generation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    color_mode='grayscale')

val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    color_mode='grayscale')

num_classes = 68

# Build the CNN model with L2 regularization
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape, kernel_regularizer=l2(0.01)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator
)

# Save the trained model
model.save("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_4.h5")

##ended this process early because it showed low accuracy and a slow growth trend



Found 6120 images belonging to 68 classes.
Found 1496 images belonging to 68 classes.
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 64, 64, 32)        320       
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 32, 32, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_14 (Conv2D)          (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 16, 16, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_15 (Conv2D)          (None, 16, 16, 128)       73856     
                                  

KeyboardInterrupt: 

In [ ]:
#M2a with increased dropout rate
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the root directory of your dataset
dataset_dir = "/content/drive/MyDrive/442final/renamed"

# Define the image size and input shape
img_height, img_width = 64, 64
input_shape = (img_height, img_width, 1)  # Assuming grayscale images

# Prepare data with augmented data generation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    color_mode='grayscale')

val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    color_mode='grayscale')

num_classes = 68

# Build the CNN model with increased dropout rates
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.6),  # Increased dropout rate for higher model regularization
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator
)

# Save the trained model
model.save("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_4.h5")


#colab is taking really long to rerun this and it got deleted


Found 6120 images belonging to 68 classes.
Found 1496 images belonging to 68 classes.
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 64, 64, 32)        320       
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 32, 32, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_23 (Conv2D)          (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 16, 16, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_24 (Conv2D)          (None, 16, 16, 128)       73856     
                                  

KeyboardInterrupt: 

In [ ]:
#test M2a with increased dropout rate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_5.h5")
model.summary()

# Set up the data generator for the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Ensure the generator matches the model input requirements
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/442final/test_set',
    target_size=(64, 64),  # Match the input size for which the model is configured
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' because your model uses categorical crossentropy
    color_mode='grayscale',  # Match the color mode to how the model was trained
    shuffle=False  # Set to False to keep data in the same order for consistency
)

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)



Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 64, 64, 32)        320       
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 32, 32, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_20 (Conv2D)          (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 16, 16, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_21 (Conv2D)          (None, 16, 16, 128)       73856     
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 8, 8, 128)        

In [ ]:
#increased drop out with larger batch sizes
#this was done just in hopes of faster computation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the root directory of your dataset
dataset_dir = "/content/drive/MyDrive/442final/renamed"

# Define the image size and input shape
img_height, img_width = 64, 64
input_shape = (img_height, img_width, 1)  # Assuming grayscale images

# Prepare data with augmented data generation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=64,
    class_mode='categorical',
    subset='training',
    color_mode='grayscale')

val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=64,
    class_mode='categorical',
    subset='validation',
    color_mode='grayscale')

num_classes = 68

# Build the CNN model with increased dropout rates
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.6),  # Increased dropout rate for higher model regularization
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator
)

# Save the trained model
model.save("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_5.h5")


Found 6120 images belonging to 68 classes.
Found 1496 images belonging to 68 classes.
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 64, 64, 32)        320       
                                                                 
 max_pooling2d_23 (MaxPooli  (None, 32, 32, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_26 (Conv2D)          (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_24 (MaxPooli  (None, 16, 16, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_27 (Conv2D)          (None, 16, 16, 128)       73856     
                                  

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#test increased dropout rate and larger batch size
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_5.h5")
model.summary()

# Set up the data generator for the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Ensure the generator matches the model input requirements
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/442final/test_set',
    target_size=(64, 64),  # Match the input size for which the model is configured
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' because your model uses categorical crossentropy
    color_mode='grayscale',  # Match the color mode to how the model was trained
    shuffle=False  # Set to False to keep data in the same order for consistency
)

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)



Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 64, 64, 32)        320       
                                                                 
 max_pooling2d_23 (MaxPooli  (None, 32, 32, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_26 (Conv2D)          (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_24 (MaxPooli  (None, 16, 16, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_27 (Conv2D)          (None, 16, 16, 128)       73856     
                                                                 
 max_pooling2d_25 (MaxPooli  (None, 8, 8, 128)        

In [ ]:
#adding residual layers to M2a with optimal dropout rates
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the root directory of your dataset
dataset_dir = "/content/drive/MyDrive/442final/renamed"

# Define the image size and input shape
img_height, img_width = 64, 64
input_shape = (img_height, img_width, 1)  # Assuming grayscale images

# Prepare data with augmented data generation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=128,
    class_mode='categorical',
    subset='training',
    color_mode='grayscale')

val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=128,
    class_mode='categorical',
    subset='validation',
    color_mode='grayscale')

num_classes = 68

# Build the CNN model with residual connections
inputs = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
ident1 = x  # Save for addition later

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Add()([x, ident1])  # Residual connection
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.6)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator
)

# Save the trained model
model.save("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_6_res.h5")

##colab wont let me run this anymore but there's proof that it exists.


In [ ]:
#test
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_6.h5")
model.summary()

# Set up the data generator for the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Ensure the generator matches the model input requirements
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/442final/renamed',
    target_size=(64, 64),  # Match the input size for which the model is configured
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' because your model uses categorical crossentropy
    color_mode='grayscale',  # Match the color mode to how the model was trained
    shuffle=False  # Set to False to keep data in the same order for consistency
)

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)



In [ ]:
#M2a increased filters and optimal dropout rate + residual layers
# Define the root directory of your dataset
dataset_dir = "/content/drive/MyDrive/442final/renamed"

# Define the image size and input shape
img_height, img_width = 64, 64
input_shape = (img_height, img_width, 1)  # Assuming grayscale images

# Prepare data with augmented data generation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=128,
    class_mode='categorical',
    subset='training',
    color_mode='grayscale')

val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=128,
    class_mode='categorical',
    subset='validation',
    color_mode='grayscale')

num_classes = 68

# Build the CNN model with increased filters and residual connections
inputs = Input(shape=input_shape)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)  # Increased from 32 to 64
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)  # Increased from 64 to 128
x = MaxPooling2D(pool_size=(2, 2))(x)
ident1 = x  # Save for addition later

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)  # Maintained at 128
x = Add()([x, ident1])  # Residual connection
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)  # Increased from 128 to 256
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)  # Increased from 256 to 512
x = Dropout(0.6)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary to see the changes
model.summary()

# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator
)

# Save the trained model
model.save("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_7.h5")

Found 6120 images belonging to 68 classes.
Found 1496 images belonging to 68 classes.
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 64, 64, 1)]          0         []                            
                                                                                                  
 conv2d_8 (Conv2D)           (None, 64, 64, 64)           640       ['input_3[0][0]']             
                                                                                                  
 max_pooling2d_8 (MaxPoolin  (None, 32, 32, 64)           0         ['conv2d_8[0][0]']            
 g2D)                                                                                             
                                                                                                  
 conv2

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
!ls #proof that at some point I did have all of these models, I just can't access them anymore for some ungodly reason

442final.ipynb						    mobilenetv3_symbol_classifier_fold_5.h5
data							    mobilenetv3_symbol_classifier_tuned.h5
mobilenetv3_symbol_classifier_2.h5			    mobilenetv3_symbol_cv_tuned.h5
mobilenetv3_symbol_classifier_3.h5			    my_dir
mobilenetv3_symbol_classifier_5.h5			    renamed
mobilenetv3_symbol_classifier_6_res_dropout_every_layer.h5  renamed_data.zip
mobilenetv3_symbol_classifier_6_res.h5			    test_set
mobilenetv3_symbol_classifier_7.h5


In [ ]:
#test model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_7.h5")
model.summary()

# Set up the data generator for the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Ensure the generator matches the model input requirements
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/442final/test_set',
    target_size=(64, 64),  # Match the input size for which the model is configured
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' because your model uses categorical crossentropy
    color_mode='grayscale',  # Match the color mode to how the model was trained
    shuffle=False  # Set to False to keep data in the same order for consistency
)

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)



Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 64, 64, 1)]          0         []                            
                                                                                                  
 conv2d_8 (Conv2D)           (None, 64, 64, 64)           640       ['input_3[0][0]']             
                                                                                                  
 max_pooling2d_8 (MaxPoolin  (None, 32, 32, 64)           0         ['conv2d_8[0][0]']            
 g2D)                                                                                             
                                                                                                  
 conv2d_9 (Conv2D)           (None, 32, 32, 128)          73856     ['max_pooling2d_8[0][0]'

In [ ]:
#added 5 fold cv
from sklearn.model_selection import KFold
import numpy as np
import os
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


# Assuming dataset_dir is your directory with images classified in subfolders for each class
image_files = []
labels = []

# Load dataset file paths and labels
for class_dir in os.listdir(dataset_dir):
    class_dir_path = os.path.join(dataset_dir, class_dir)
    if os.path.isdir(class_dir_path):
        for img in os.listdir(class_dir_path):
            image_files.append(os.path.join(class_dir_path, img))
            labels.append(class_dir)  # assuming folder names are class labels

image_files = np.array(image_files)
labels = np.array(labels)

# Define the K-Folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# This will print out the indices for each fold
for train_index, val_index in kf.split(image_files):
    train_images, val_images = image_files[train_index], image_files[val_index]
    train_labels, val_labels = labels[train_index], labels[val_index]

    # Now create a data generator for training
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    val_datagen = ImageDataGenerator(rescale=1./255)

    # Flow from directory expects directories for each class with images inside,
    # so you need to create a temporary directory structure or use flow_from_dataframe

    # Assuming the use of flow_from_dataframe if directories are not to be created
    train_generator = train_datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': train_images, 'class': train_labels}),
        directory=None,  # no directory structure used
        x_col='filename',
        y_col='class',
        target_size=(img_height, img_width),
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=128,
        shuffle=True
    )

    val_generator = val_datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': val_images, 'class': val_labels}),
        directory=None,  # no directory structure used
        x_col='filename',
        y_col='class',
        target_size=(img_height, img_width),
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=128,
        shuffle=False
    )

    # Build, compile, and fit your model inside the loop
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    # Fit the model
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, min_lr=0.0001)

    # Train the model with callbacks
    history = model.fit(
        train_generator,
        epochs=20,
        validation_data=val_generator,
        callbacks=[early_stopping, reduce_lr]
    )

    # Optionally save the model
    model.save(f"/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_fold_5.h5")


Found 6092 validated image filenames belonging to 68 classes.
Found 1524 validated image filenames belonging to 68 classes.
Epoch 1/20
48/48 [==============================] - 20s 371ms/step - loss: 1.0807 - accuracy: 0.6037 - val_loss: 0.6776 - val_accuracy: 0.7238 - lr: 0.0010
Epoch 2/20
48/48 [==============================] - 17s 364ms/step - loss: 1.0582 - accuracy: 0.6080 - val_loss: 0.8447 - val_accuracy: 0.6693 - lr: 0.0010
Epoch 3/20
48/48 [==============================] - 18s 369ms/step - loss: 1.0337 - accuracy: 0.6193 - val_loss: 0.7233 - val_accuracy: 0.7257 - lr: 0.0010
Epoch 4/20
48/48 [==============================] - 17s 365ms/step - loss: 1.0168 - accuracy: 0.6300 - val_loss: 0.7805 - val_accuracy: 0.6975 - lr: 0.0010
Epoch 5/20
48/48 [==============================] - 18s 366ms/step - loss: 1.0463 - accuracy: 0.6124 - val_loss: 0.8777 - val_accuracy: 0.6654 - lr: 0.0010
Epoch 6/20
48/48 [==============================] - 18s 378ms/step - loss: 1.0234 - accuracy: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 6093 validated image filenames belonging to 68 classes.
Found 1523 validated image filenames belonging to 68 classes.
Epoch 1/20
48/48 [==============================] - 19s 366ms/step - loss: 1.0629 - accuracy: 0.6202 - val_loss: 0.8416 - val_accuracy: 0.6756 - lr: 0.0010
Epoch 2/20
48/48 [==============================] - 18s 372ms/step - loss: 1.0206 - accuracy: 0.6268 - val_loss: 0.7438 - val_accuracy: 0.7006 - lr: 0.0010
Epoch 3/20
48/48 [==============================] - 18s 371ms/step - loss: 0.9903 - accuracy: 0.6245 - val_loss: 0.7306 - val_accuracy: 0.7183 - lr: 0.0010
Epoch 4/20
48/48 [==============================] - 18s 373ms/step - loss: 1.0152 - accuracy: 0.6315 - val_loss: 0.9246 - val_accuracy: 0.6520 - lr: 0.0010
Epoch 5/20
48/48 [==============================] - 17s 364ms/step - loss: 0.9975 - accuracy: 0.6398 - val_loss: 0.7712 - val_accuracy: 0.6914 - lr: 0.0010
Epoch 6/20
48/48 [==============================] - 17s 361ms/step - loss: 1.0092 - accuracy: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 6093 validated image filenames belonging to 68 classes.
Found 1523 validated image filenames belonging to 68 classes.
Epoch 1/20
48/48 [==============================] - 20s 374ms/step - loss: 1.0296 - accuracy: 0.6220 - val_loss: 0.8263 - val_accuracy: 0.6868 - lr: 0.0010
Epoch 2/20
48/48 [==============================] - 17s 362ms/step - loss: 1.0196 - accuracy: 0.6243 - val_loss: 0.7925 - val_accuracy: 0.6861 - lr: 0.0010
Epoch 3/20
48/48 [==============================] - 19s 387ms/step - loss: 0.9960 - accuracy: 0.6373 - val_loss: 0.8034 - val_accuracy: 0.6861 - lr: 0.0010
Epoch 4/20
48/48 [==============================] - 18s 366ms/step - loss: 0.9972 - accuracy: 0.6389 - val_loss: 0.7592 - val_accuracy: 0.7006 - lr: 0.0010
Epoch 5/20
48/48 [==============================] - 18s 369ms/step - loss: 0.9723 - accuracy: 0.6402 - val_loss: 0.7329 - val_accuracy: 0.7098 - lr: 0.0010
Epoch 6/20
48/48 [==============================] - 18s 369ms/step - loss: 0.9839 - accuracy: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 6093 validated image filenames belonging to 68 classes.
Found 1523 validated image filenames belonging to 68 classes.
Epoch 1/20
48/48 [==============================] - 20s 371ms/step - loss: 1.0272 - accuracy: 0.6266 - val_loss: 0.6734 - val_accuracy: 0.7150 - lr: 0.0010
Epoch 2/20
48/48 [==============================] - 18s 372ms/step - loss: 0.9876 - accuracy: 0.6343 - val_loss: 0.7200 - val_accuracy: 0.6947 - lr: 0.0010
Epoch 3/20
48/48 [==============================] - 18s 379ms/step - loss: 0.9798 - accuracy: 0.6309 - val_loss: 0.7654 - val_accuracy: 0.6947 - lr: 0.0010
Epoch 4/20
48/48 [==============================] - 18s 368ms/step - loss: 0.9861 - accuracy: 0.6424 - val_loss: 0.8581 - val_accuracy: 0.6678 - lr: 0.0010
Epoch 5/20
48/48 [==============================] - 18s 365ms/step - loss: 0.9600 - accuracy: 0.6493 - val_loss: 0.8552 - val_accuracy: 0.6638 - lr: 0.0010
Epoch 6/20
48/48 [==============================] - 18s 368ms/step - loss: 0.9839 - accuracy: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 6093 validated image filenames belonging to 68 classes.
Found 1523 validated image filenames belonging to 68 classes.
Epoch 1/20
48/48 [==============================] - 19s 365ms/step - loss: 1.0420 - accuracy: 0.6176 - val_loss: 0.7611 - val_accuracy: 0.6875 - lr: 0.0010
Epoch 2/20
48/48 [==============================] - 18s 379ms/step - loss: 1.0128 - accuracy: 0.6309 - val_loss: 0.6718 - val_accuracy: 0.7058 - lr: 0.0010
Epoch 3/20
48/48 [==============================] - 17s 361ms/step - loss: 0.9854 - accuracy: 0.6425 - val_loss: 0.7679 - val_accuracy: 0.6947 - lr: 0.0010
Epoch 4/20
48/48 [==============================] - 18s 369ms/step - loss: 0.9862 - accuracy: 0.6312 - val_loss: 0.6658 - val_accuracy: 0.7229 - lr: 0.0010
Epoch 5/20
48/48 [==============================] - 18s 370ms/step - loss: 0.9919 - accuracy: 0.6402 - val_loss: 0.6956 - val_accuracy: 0.7111 - lr: 0.0010
Epoch 6/20
48/48 [==============================] - 18s 368ms/step - loss: 0.9526 - accuracy: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
#test
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_fold_5.h5")
model.summary()

# Set up the data generator for the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Ensure the generator matches the model input requirements
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/442final/test_set',
    target_size=(64, 64),  # Match the input size for which the model is configured
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' because your model uses categorical crossentropy
    color_mode='grayscale',  # Match the color mode to how the model was trained
    shuffle=False  # Set to False to keep data in the same order for consistency
)

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
##this is my best model so far


Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 64, 64, 1)]          0         []                            
                                                                                                  
 conv2d_8 (Conv2D)           (None, 64, 64, 64)           640       ['input_3[0][0]']             
                                                                                                  
 max_pooling2d_8 (MaxPoolin  (None, 32, 32, 64)           0         ['conv2d_8[0][0]']            
 g2D)                                                                                             
                                                                                                  
 conv2d_9 (Conv2D)           (None, 32, 32, 128)          73856     ['max_pooling2d_8[0][0]'

In [ ]:
#add dropout layer after every convolution layer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define the root directory of your dataset
dataset_dir = "/content/drive/MyDrive/442final/renamed"

# Define the image size and input shape
img_height, img_width = 64, 64
input_shape = (img_height, img_width, 1)  # Assuming grayscale images

# Prepare data with augmented data generation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=128,
    class_mode='categorical',
    subset='training',
    color_mode='grayscale')

val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=128,
    class_mode='categorical',
    subset='validation',
    color_mode='grayscale')

num_classes = 68

# CNN model with dropout layers after each convolutional layer
inputs = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = Dropout(0.3)(x)  # Adding dropout
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Dropout(0.3)(x)  # Adding dropout
x = MaxPooling2D(pool_size=(2, 2))(x)
ident1 = x  # Save for addition later

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Dropout(0.3)(x)  # Adding dropout
x = Add()([x, ident1])  # Residual connection
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = Dropout(0.4)(x)  # Increased dropout for deeper layers
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

# Early Stopping and Reduce LR On Plateau Callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, min_lr=0.0001)

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Train
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)

# Save the trained model
model.save("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_6_res_dropout_every_layer.h5")


Found 6120 images belonging to 68 classes.
Found 1496 images belonging to 68 classes.
Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 64, 64, 1)]          0         []                            
                                                                                                  
 conv2d_26 (Conv2D)          (None, 64, 64, 32)           320       ['input_7[0][0]']             
                                                                                                  
 dropout_10 (Dropout)        (None, 64, 64, 32)           0         ['conv2d_26[0][0]']           
                                                                                                  
 max_pooling2d_22 (MaxPooli  (None, 32, 32, 32)           0         ['dropout_10[0][0]']          
 ng2D

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#test
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_6_res_dropout_every_layer.h5")
model.summary()

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/442final/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale',
    shuffle=False  # Set to False to keep data in the same order for consistency
)

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)



Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 64, 64, 1)]          0         []                            
                                                                                                  
 conv2d_26 (Conv2D)          (None, 64, 64, 32)           320       ['input_7[0][0]']             
                                                                                                  
 dropout_10 (Dropout)        (None, 64, 64, 32)           0         ['conv2d_26[0][0]']           
                                                                                                  
 max_pooling2d_22 (MaxPooli  (None, 32, 32, 32)           0         ['dropout_10[0][0]']          
 ng2D)                                                                                     

In [ ]:
pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.0 MB/s eta 0:00:00


In [ ]:
#the more filters M2a with keras tuning for hyper params
import kerastuner as kt
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def build_model(hp):
    # Hyperparameters
    num_filters = hp.Int('num_filters', min_value=32, max_value=256, step=32)
    kernel_size = hp.Choice('kernel_size', values=[3, 5])
    dropout_rate = hp.Float('dropout_rate', min_value=0.0, max_value=0.6, step=0.1)

    inputs = Input(shape=(64, 64, 1))
    x = Conv2D(num_filters, (kernel_size, kernel_size), activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(num_filters * 2, (kernel_size, kernel_size), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    ident1 = x
    x = Conv2D(num_filters * 2, (kernel_size, kernel_size), activation='relu', padding='same')(x)
    x = Add()([x, ident1])
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(num_filters * 4, (kernel_size, kernel_size), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(68, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Prepare data
datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
                             shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest', validation_split=0.2)

train_generator = datagen.flow_from_directory(dataset_dir, target_size=(64, 64), batch_size=128,
                                              class_mode='categorical', subset='training', color_mode='grayscale')
val_generator = datagen.flow_from_directory(dataset_dir, target_size=(64, 64), batch_size=128,
                                            class_mode='categorical', subset='validation', color_mode='grayscale')

# Setup the tuner
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

# Search for the best hyperparameters
tuner.search(train_generator, epochs=50, validation_data=val_generator, callbacks=[EarlyStopping(monitor='val_accuracy', patience=5)])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
model.fit(train_generator, epochs=50, validation_data=val_generator)


Trial 30 Complete [00h 03m 31s]
val_accuracy: 0.48262032866477966

Best val_accuracy So Far: 0.5387700796127319
Total elapsed time: 00h 42m 02s
Epoch 1/50
48/48 [==============================] - 20s 388ms/step - loss: 3.9901 - accuracy: 0.0392 - val_loss: 3.6233 - val_accuracy: 0.0715
Epoch 2/50
48/48 [==============================] - 19s 398ms/step - loss: 3.2719 - accuracy: 0.1199 - val_loss: 2.6718 - val_accuracy: 0.2106
Epoch 3/50
48/48 [==============================] - 19s 389ms/step - loss: 2.6456 - accuracy: 0.2297 - val_loss: 2.2141 - val_accuracy: 0.3048
Epoch 4/50
48/48 [==============================] - 18s 380ms/step - loss: 2.2151 - accuracy: 0.3142 - val_loss: 1.8504 - val_accuracy: 0.3864
Epoch 5/50
48/48 [==============================] - 19s 389ms/step - loss: 1.9639 - accuracy: 0.3680 - val_loss: 1.6923 - val_accuracy: 0.4465
Epoch 6/50
48/48 [==============================] - 20s 420ms/step - loss: 1.7825 - accuracy: 0.4167 - val_loss: 1.5874 - val_accuracy: 0.467

In [ ]:
model.save("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_tuned.h5")
model.summary()



Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 64, 64, 1)]          0         []                            
                                                                                                  
 conv2d_4 (Conv2D)           (None, 64, 64, 64)           1664      ['input_2[0][0]']             
                                                                                                  
 max_pooling2d_4 (MaxPoolin  (None, 32, 32, 64)           0         ['conv2d_4[0][0]']            
 g2D)                                                                                             
                                                                                                  
 conv2d_5 (Conv2D)           (None, 32, 32, 128)          204928    ['max_pooling2d_4[0][0]'

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Total params: 3568324 (13.61 MB)
Trainable params: 3568324 (13.61 MB)
Non-trainable params: 0 (0.00 Byte)
__________________________________________________________________________________________________


In [ ]:
#test
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_tuned.h5")
model.summary()

# Set up the data generator for the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Ensure the generator matches the model input requirements
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/442final/test_set',
    target_size=(64, 64),  # Match the input size for which the model is configured
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' because your model uses categorical crossentropy
    color_mode='grayscale',  # Match the color mode to how the model was trained
    shuffle=False  # Set to False to keep data in the same order for consistency
)

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)



Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 64, 64, 1)]          0         []                            
                                                                                                  
 conv2d_4 (Conv2D)           (None, 64, 64, 64)           1664      ['input_2[0][0]']             
                                                                                                  
 max_pooling2d_4 (MaxPoolin  (None, 32, 32, 64)           0         ['conv2d_4[0][0]']            
 g2D)                                                                                             
                                                                                                  
 conv2d_5 (Conv2D)           (None, 32, 32, 128)          204928    ['max_pooling2d_4[0][0]'

In [ ]:
#keras tuned model as base k-fold c.v
from sklearn.model_selection import KFold
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model = load_model('/content/drive/MyDrive/442final/mobilenetv3_symbol_classifier_tuned.h5')

# Set up K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

results = []

# Assuming dataset_dir is your directory with images classified in subfolders for each class
image_files = []
labels = []

# Load dataset file paths and labels
for class_dir in os.listdir(dataset_dir):
    class_dir_path = os.path.join(dataset_dir, class_dir)
    if os.path.isdir(class_dir_path):
        for img in os.listdir(class_dir_path):
            image_files.append(os.path.join(class_dir_path, img))
            labels.append(class_dir)  # assuming folder names are class labels

image_files = np.array(image_files)
labels = np.array(labels)

for train_index, val_index in kf.split(image_files):
    train_images, val_images = image_files[train_index], image_files[val_index]
    train_labels, val_labels = labels[train_index], labels[val_index]

    # Prepare training and validation data using flow_from_dataframe
    train_data = pd.DataFrame({
        'filename': train_images,
        'class': train_labels
    })

    val_data = pd.DataFrame({
        'filename': val_images,
        'class': val_labels
    })

    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    train_generator = datagen.flow_from_dataframe(
        dataframe=train_data,
        x_col='filename',
        y_col='class',
        target_size=(img_height, img_width),
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=128,
        shuffle=True
    )

    val_generator = datagen.flow_from_dataframe(
        dataframe=val_data,
        x_col='filename',
        y_col='class',
        target_size=(img_height, img_width),
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=128,
        shuffle=False
    )

    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, min_lr=0.0001)

    # Train your model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(
        train_generator,
        epochs=20,
        validation_data=val_generator,
        callbacks=[early_stopping, reduce_lr]
    )

    results.append(history.history)

# Output or examine your results
print(results)


Found 6092 validated image filenames belonging to 68 classes.
Found 1524 validated image filenames belonging to 68 classes.
Epoch 1/20
48/48 [==============================] - 21s 389ms/step - loss: 0.8842 - accuracy: 0.6564 - val_loss: 0.7796 - val_accuracy: 0.6975 - lr: 0.0010
Epoch 2/20
48/48 [==============================] - 18s 386ms/step - loss: 0.8405 - accuracy: 0.6793 - val_loss: 0.8056 - val_accuracy: 0.6870 - lr: 0.0010
Epoch 3/20
48/48 [==============================] - 19s 391ms/step - loss: 0.8240 - accuracy: 0.6811 - val_loss: 0.8194 - val_accuracy: 0.6929 - lr: 0.0010
Epoch 4/20
48/48 [==============================] - 18s 384ms/step - loss: 0.8055 - accuracy: 0.6935 - val_loss: 0.8207 - val_accuracy: 0.6870 - lr: 0.0010
Epoch 5/20
48/48 [==============================] - 19s 393ms/step - loss: 0.8137 - accuracy: 0.6861 - val_loss: 0.7977 - val_accuracy: 0.6949 - lr: 0.0010
Epoch 6/20
48/48 [==============================] - 18s 386ms/step - loss: 0.8114 - accuracy: 0.

In [ ]:
model.save('/content/drive/MyDrive/442final/mobilenetv3_symbol_cv_tuned.h5')
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 64, 64, 1)]          0         []                            
                                                                                                  
 conv2d_4 (Conv2D)           (None, 64, 64, 64)           1664      ['input_2[0][0]']             
                                                                                                  
 max_pooling2d_4 (MaxPoolin  (None, 32, 32, 64)           0         ['conv2d_4[0][0]']            
 g2D)                                                                                             
                                                                                                  
 conv2d_5 (Conv2D)           (None, 32, 32, 128)          204928    ['max_pooling2d_4[0][0]'

In [ ]:
#test
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/442final/mobilenetv3_symbol_cv_tuned.h5")
model.summary()

# Set up the data generator for the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Ensure the generator matches the model input requirements
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/442final/test_set',
    target_size=(64, 64),  # Match the input size for which the model is configured
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' because your model uses categorical crossentropy
    color_mode='grayscale',  # Match the color mode to how the model was trained
    shuffle=False  # Set to False to keep data in the same order for consistency
)

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
##barely an improvement, did not mention in report


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 64, 64, 1)]          0         []                            
                                                                                                  
 conv2d_4 (Conv2D)           (None, 64, 64, 64)           1664      ['input_2[0][0]']             
                                                                                                  
 max_pooling2d_4 (MaxPoolin  (None, 32, 32, 64)           0         ['conv2d_4[0][0]']            
 g2D)                                                                                             
                                                                                                  
 conv2d_5 (Conv2D)           (None, 32, 32, 128)          204928    ['max_pooling2d_4[0][0]'

In [11]:
!ls #all these models existed at some point I swear I don't know why google drive wont let me have them

442final.ipynb						    mobilenetv3_symbol_classifier_fold_5.h5
data							    mobilenetv3_symbol_classifier_tuned.h5
mobilenetv3_symbol_classifier_2.h5			    mobilenetv3_symbol_cv_tuned.h5
mobilenetv3_symbol_classifier_3.h5			    my_dir
mobilenetv3_symbol_classifier_5.h5			    renamed
mobilenetv3_symbol_classifier_6_res_dropout_every_layer.h5  renamed_data.zip
mobilenetv3_symbol_classifier_6_res.h5			    test_set
mobilenetv3_symbol_classifier_7.h5
